<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/Milena/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install at every runtime?-prob yes
!pip install checklist
!pip install torch

#should i install it every time?-yes
!pip install simpletransformers 


     |████████████████████████████████| 12.1 MB 14.4 MB/s 
     |████████████████████████████████| 4.2 MB 52.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.6 MB/s 
     |████████████████████████████████| 167 kB 65.7 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 
     |████████████████████████████████| 596 kB 61.8 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 5.6 MB 61.6 MB/s 
     |████████████████████████████████| 5.6 MB 59.7 MB/s 
     |████████████████████████████████| 419 kB 61.7 MB/s 
     |████████████████████████████████| 104 kB 72.3 MB/s 
     |████████████████████████████████| 3.6 MB 51.7 MB/s 
  Created wheel for checklist: filename=checklist-0.0.11-py3-none-any.whl size=12165635 sha256=edca4ec2b72302ba8a388e0a2b3ea9285e88b9b5dcece78152d6b29e2e17d6b1
  Stored in directory: /root/.cache/pip/wheels/6a/8a/07/6446879be434879c27671c83443727d7

In [1]:
import checklist
import torch
import spacy
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#$loading eng
nlp = spacy.load("en_core_web_sm")

In [2]:
# from platform import python_version #to check version
# python_version()

import pandas as pd
import checklist
from checklist.perturb import Perturb
#from checklist.perturb import add_negation
import nltk #; nltk.download('omw-1.4')
#nltk.download()

In [3]:
# from E6
#setting up google drive and loading diagnostic test data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv")
url = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
#data = pd.read_csv(url)

#$loading eng
#nlp = spacy.load("en_core_web_sm")
print(data)

Mounted at /content/drive
       id                                               text  labels
0   89200      @USER @USER Who the hell does he think he is?       1
1   71294  #BREAKING. #Greece: Molotov cocktails fly afte...       1
2   55633  #OrrinHatch I can’t believe this sexist , clue...       1
3   16856  @USER @USER I'll use that one the next time im...       1
4   26072    0-1 lost my acca on the first fucking fight cba       1
..    ...                                                ...     ...
95  45518  @USER He is obviously getting suspended. He is...       0
96  51610  #Canada - EXCLUSIVE: #Trudeau #Liberals leave ...       0
97  26758  @USER @USER ...than why did you show us how ho...       0
98  30718  @USER @USER @USER You have yet to answer what ...       0
99  82558  #MAGA  ... got any ideas how she could have do...       0

[100 rows x 3 columns]


In [4]:
#loading finetuned MODEL FROM DRIVE
bert_mod = ClassificationModel("bert", "/content/drive/MyDrive/NLP2/trained_models3")

In [5]:
np.random.seed(42)

#adding typos from tutorial
from checklist.perturb import Perturb

pdata = list(data['text'])
labels = list(data['labels'])
ret = Perturb.perturb(pdata, Perturb.add_typos)
ret.data 

[['@USER @USER Who the hell does he think he is?',
  '@USER @USER Who the hell does he thinkh e is?'],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  '#BREAKING. #Greece: Molotov cocktails fly after proetst honouring killed antifa arti... URL via @USER URL'],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges aaginst a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can 

In [6]:
#keeping only the typos and recreating them into a df
typo_lst=[]
for i in range(len(ret.data)):
  typo_lst.append(ret.data[i][1])

#(typo_lst)
typo_df = pd.DataFrame(list(zip(typo_lst, labels)), columns =['text', 'label'])
print(typo_df)

                                                 text  label
0       @USER @USER Who the hell does he thinkh e is?      1
1   #BREAKING. #Greece: Molotov cocktails fly afte...      1
2   #OrrinHatch I can’t believe this sexist , clue...      1
3   @USER @USER I'll use that one the next time im...      1
4     0-1 lost my acca on hte first fucking fight cba      1
..                                                ...    ...
95  @USER H eis obviously getting suspended. He is...      0
96  #Canada - EXCLUSIVE: #Trudeau #Libreals leave ...      0
97  @USER @USER ...than why did you shwo us how ho...      0
98  @USER @USER @USER You have yet t oanswer what ...      0
99  #MAG A ... got any ideas how she could have do...      0

[100 rows x 2 columns]


In [7]:
#geting predictions on the typos
prediction_typo, out_nr_typo = bert_mod.predict(typo_lst)
prediction_reg, out_nr_reg = bert_mod.predict(pdata)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
result, model_outputs, wrong_predictions = bert_mod.eval_model(data, clf_report = classification_report)
result_typo, model_outputs_typo, wrong_predictions_typo = bert_mod.eval_model(typo_df, clf_report = classification_report)

print(result['clf_report'])
print(result_typo['clf_report'])

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.78      0.90      0.83        50
         1.0       0.88      0.74      0.80        50

    accuracy                           0.82       100
   macro avg       0.83      0.82      0.82       100
weighted avg       0.83      0.82      0.82       100

              precision    recall  f1-score   support

         0.0       0.76      0.94      0.84        50
         1.0       0.92      0.70      0.80        50

    accuracy                           0.82       100
   macro avg       0.84      0.82      0.82       100
weighted avg       0.84      0.82      0.82       100



In [10]:
# quantitative analysis
def precision(true_pos, false_pos):
    if true_pos + false_pos != 0:
        prec = true_pos/(true_pos + false_pos)
    else:
        prec = 0
    return prec

def recall(true_pos, false_neg):
    if true_pos + false_neg != 0:
        rec = true_pos/(true_pos + false_neg)
    else:
        rec = 0
    return rec

def f1(precision, recall):
    if precision + recall != 0:
        f1_score = 2*((precision * recall)/(precision + recall))
    else:
        f1_score = 0
    return f1_score

In [11]:
# this shows agreement of the two methods
pred0_acc0 = 0
pred1_acc0 = 0
pred0_acc1 = 0
pred1_acc1 = 0

for i in range(len(typo_lst)):
  if prediction_reg[i] == 0 and prediction_typo[i] == 0:
    pred0_acc0 += 1
  elif prediction_reg[i] == 1 and prediction_typo[i] == 0:
    pred0_acc1 += 1
  elif prediction_reg[i] == 0 and prediction_typo[i] == 1:
    pred1_acc0 += 1
  elif prediction_reg[i] == 1 and prediction_typo[i] == 1:
    pred1_acc1 += 1

#print(pred0_acc0, pred1_acc1, pred0_acc1, pred1_acc1)

precision_0 = precision(pred0_acc0, pred0_acc1)
recall_0 = recall(pred0_acc0, pred1_acc0)
f1_0 = f1(precision_0, recall_0)
print(precision_0, recall_0, f1_0)

precision_1 = precision(pred1_acc1, pred1_acc0)
recall_1 = recall(pred1_acc1, pred0_acc1)
f1_1 = f1(precision_1, recall_1)
print(precision_1, recall_1, f1_1)

macro_precision = (precision_0+precision_1)/2
macro_recall = (recall_0 + recall_1)/2
macro_f1 = (f1_0 + f1_1)/2
print(macro_precision, macro_recall, macro_f1)

0.9193548387096774 0.9827586206896551 0.95
0.9736842105263158 0.8809523809523809 0.925
0.9465195246179966 0.9318555008210181 0.9375


In [21]:
# show whole df
pd.set_option('display.max_rows', 1000)

summary = pd.DataFrame(list(zip(typo_lst, prediction_typo, prediction_reg, labels)), columns = ['text', 'typo', 'original', 'label'])
print(summary)

                                                 text  typo  original  label
0       @USER @USER Who the hell does he thinkh e is?     1         1      1
1   #BREAKING. #Greece: Molotov cocktails fly afte...     0         0      1
2   #OrrinHatch I can’t believe this sexist , clue...     1         1      1
3   @USER @USER I'll use that one the next time im...     1         0      1
4     0-1 lost my acca on hte first fucking fight cba     1         1      1
5   #Bakersfield is why we need gun control! Sorry...     1         1      1
6   #Christian #America – If we go by #Trump’s exa...     0         0      1
7   @USER @USER @USER She is the most disingenuous...     1         1      1
8   #Democrats #Liberals you are being #threatened...     0         0      1
9   699. Just wanted to tell you you should Hang O...     1         1      1
10  #LiberalHypocrisy #criminal #democrat #evidenc...     1         1      1
11                (cr1tikal voice) smef m yass cheeks     0         1      1

In [22]:
# how many sentences did the model misclassify?
counter = 0

for i in range(len(typo_lst)):
  if prediction_reg[i] == 0 and labels[i] == 1:
    counter += 1

print(counter)

13
